# Entrenamiento de Behavioral Cloning CNN
Este notebook entrena una red neuronal convolucional para predecir ángulos de dirección a partir de imágenes capturadas en Webots.

In [1]:
# Clonar el repositorio con las imágenes y CSV
!git clone https://github.com/juliomestas/navegacion_autonoma.git

Cloning into 'navegacion_autonoma'...
remote: Enumerating objects: 56787, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 56787 (delta 0), reused 0 (delta 0), pack-reused 56782 (from 2)
Receiving objects: 100% (56787/56787), 1.44 GiB | 53.83 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (36128/36128), done.


In [2]:
# Librerías necesarias
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Lambda
from tensorflow.keras.utils import img_to_array
from sklearn.model_selection import train_test_split

In [3]:
# Cargar imágenes y ángulos
def load_data(base_path):
    df = pd.read_csv(os.path.join(base_path, 'angles.csv'))
    X, y = [], []
    for _, row in df.iterrows():
        img_path = os.path.join(base_path, 'captured_images', row['filename'])
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (200, 66))
            X.append(img_to_array(img))
            y.append(row['vision_angle'])
    return np.array(X), np.array(y)

# Cargar ambos recorridos y combinarlos
X1, y1 = load_data('navegacion_autonoma/controllers/my_vehicle_controller/circuito_ida_con_trapecio')
X2, y2 = load_data('navegacion_autonoma/controllers/my_vehicle_controller/circuito_regreso_con_trapecio')

X = np.concatenate([X1, X2]) / 255.0
y = np.concatenate([y1, y2])

In [4]:
# Data augmentation (flip horizontal)
X_flipped = np.array([np.fliplr(img) for img in X])
y_flipped = -y
X = np.concatenate([X, X_flipped])
y = np.concatenate([y, y_flipped])

In [5]:
# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Modelo CNN estilo NVIDIA
model = Sequential([
    Lambda(lambda x: x, input_shape=(66, 200, 3)),
    Conv2D(24, (5,5), strides=(2,2), activation='relu'),
    Conv2D(36, (5,5), strides=(2,2), activation='relu'),
    Conv2D(48, (5,5), strides=(2,2), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ (None, 66, 200, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 31, 98, 24)     │         1,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 47, 36)     │        21,636 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 5, 22, 48)      │        43,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 3, 20, 64)      │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 1, 18, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │       115,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 252,219 (985.23 KB)

 Trainable params: 252,219 (985.23 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Entrenamiento
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - loss: 0.0056 - val_loss: 0.0034
Epoch 2/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0033 - val_loss: 0.0034
Epoch 3/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0031 - val_loss: 0.0032
Epoch 4/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0029 - val_loss: 0.0031
Epoch 5/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 6/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0027 - val_loss: 0.0031
Epoch 7/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0027 - val_loss: 0.0030
Epoch 8/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0026 - val_loss: 0.0030
Epoch 9/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0026 - val_loss: 0.0030
Epoch 10/10
1615/1615 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0025 - val_loss: 0.0029


In [ ]:
# Guardar modelo
model.save('model_circuito.h5')